
# Objective
- Unstructured text data --> normalized word list
- Extract feature from text data

# Method
- Number to Words Translation
- Case to Lower
- Emoticon to Words Translation
- Symbol to Words Translation
- Translate Reduplication
- Non-Alphanumeric removal
- Slang and Abbrv Lookup
- Word Standardization
- Tokenization

In [1]:
import pandas as pd
import numpy as np
import re
import string
import itertools
import collections
from collections import OrderedDict, Counter

In [2]:
raw = pd.read_csv('data/data/data_raw.csv')
raw

,rating,review,aspect_category,polarity
0,3,Sangat Membantu. Terutama di kondisi Pandemi. ...,Pelayanan,Negatif
1,5,Baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,1,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,5,Terimakasih Halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,1,Ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif
...,...,...,...,...
795,5,"Semudah diklik, praktis dan sangat membantu👌",Anekdot/Serbaneka,Positif
796,5,Pelayanannya ramah . Paham dengan bidangnya,Anekdot/Serbaneka,Positif
797,4,bagus aplikasi nya terasa punya dokter pribadi...,Anekdot/Serbaneka,Positif
798,5,sangat memudahkan konsultasi tanpa keluar rumah,Pelayanan,Positif


# Contoh Data untuk Cleaning

# Import Library untuk Filtering

In [3]:
stop_words = np.array(pd.read_csv("data/external/stopwords_ID.txt",
                        sep="\n", header=None).values)
neg_words = np.array(pd.read_csv("data/external/negative_keyword_ID.txt",
                        sep="\n", header=None).values)
pos_words = np.array(pd.read_csv("data/external/positive_keyword_ID.txt",
                        sep="\n", header=None).values)
slang_words = pd.read_csv("data/external/kbba_ID.txt",
                        sep="\t", header=None)
root_words = np.array(pd.read_csv("data/external/rootword_ID.txt",
                        sep="\n", header=None).values)
slang = pd.read_csv("data/external/slangword_ID.txt",
                        sep=":", header=None)
emoticon = pd.read_csv("data/external/emoticon.txt",
                        sep="\t", header=None)
booster_words = np.array(pd.read_csv("data/external/boosterword_ID.txt",
                        sep="\n", header=None).values)
baku_words = pd.read_csv("data/external/katabaku_ID.txt",
                        sep="|", header=None)
baku_words.columns = [1,0]

In [4]:
slang_words = pd.concat([slang_words, slang, baku_words])
sentiment_words = np.concatenate((pos_words, neg_words, booster_words))
slang_words.drop_duplicates(inplace=True)
emoticon.drop_duplicates(inplace=True)
emoticon = dict(zip(emoticon[0], emoticon[1]))
slang_words = dict(zip(slang_words[0],slang_words[1]))
neg_words = np.unique(neg_words)
pos_words = np.unique(pos_words)
stop_words = np.unique(stop_words)
stop_words = [word for word in stop_words if word not in sentiment_words]

In [5]:
print("------Dictionary Info---------")
print("Slang words = "  + str(len(slang_words)) + " entries")
print("Emoticon = "  + str(len(emoticon)) + " entries" )
print("Root words = "  + str(len(root_words)) + " entries")
print("Sentiment words = "  + str(len(sentiment_words)) + " entries")

------Dictionary Info---------
Slang words = 2732 entries
Emoticon = 173 entries
Root words = 27979 entries
Sentiment words = 8449 entries


In [6]:
satuan = ['', 'satu', 'dua', 'tiga', 'empat', 'lima', 'enam', 'tujuh',
          'delapan', 'sembilan', 'sepuluh', 'sebelas']
    
def terbilang_(n):
    if n >= 0 and n <= 11:
        hasil = [satuan[n]]
    elif n >= 12 and n <= 19:
        hasil = terbilang_(n % 10) + ['belas']
    elif n >= 20 and n <= 99:
        hasil = terbilang_(n // 10) + ['puluh'] + terbilang_(n % 10)
    elif n >= 100 and n <= 199:
        hasil = ['seratus'] + terbilang_(n - 100)
    elif n >= 200 and n <= 999:
        hasil = terbilang_(n // 100) + ['ratus'] + terbilang_(n % 100)
    elif n >= 1000 and n <= 1999:
        hasil = ['seribu'] + terbilang_(n - 1000)
    elif n >= 2000 and n <= 999999:
        hasil = terbilang_(n // 1000) + ['ribu'] + terbilang_(n % 1000)
    elif n >= 1000000 and n <= 999999999:
        hasil = terbilang_(n // 1000000) + ['juta'] + terbilang_(n % 1000000)
    else:
        hasil = terbilang_(n // 1000000000) + ['milyar'] + terbilang_(n % 100000000)
    return hasil

def terbilang(n):
    if n == 0:
        return 'nol'
    t = terbilang_(n)
    while '' in t:
        t.remove('')
    return ' '.join(t)

def translate_angka(review):
    number = re.findall("[0-9]+",review)
    for digit in number:
        review = re.sub(digit,terbilang(int(digit))+" ",review)
    return review

def translate_repeating_words(review):
    repeating_words = re.findall(r'\w*(?:2|")',review)
    for word in repeating_words:
        cleaned = word[:-1]
        review = re.sub(word,cleaned + " " + cleaned, review)
    return review

#def delete_suffix_nya(review):
    #return re.sub("(?:nya|ny)[$|\s]"," ",review)

def translate_emoticon(t):
    for w, v in emoticon.items():
        pattern = re.compile(re.escape(w))
        match = re.search(pattern,t)
        if match:
            t = re.sub(pattern,v,t)
    return t

def translate_non_alpha_num(t):
    non_alpha_num = {
        '%' : 'persen',
        '$' : 'dolar',
        '@' : 'di',
        '&' : 'dan',
        '/' : 'atau',
        '+' : 'plus'
    }
    for w, v in non_alpha_num.items():
        pattern = re.compile(re.escape(w))
        match = re.search(pattern,t)
        if match:
            t = re.sub(pattern,v + " ",t)
    return t

def remove_non_alphanumeric(review):
    return re.sub("[^a-zA-Z\d]"," ", review)

def normalizing_words(review):
    return ''.join(''.join(s)[:1] for _, s in itertools.groupby(review))

In [7]:
raw_clean = raw

# Filtering 

In [8]:
raw_clean['review'] = raw_clean['review'].apply(translate_repeating_words)
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,Sangat Membantu. Terutama di kondisi Pandemi. ...,Pelayanan,Negatif
1,5,Baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,1,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,5,Terimakasih Halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,1,Ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


In [9]:
raw_clean['review'][662]

'Udah  x konsultasi disini, semoga selalu menjadi baik'

In [10]:
raw_clean['review'] = raw_clean['review'].apply(translate_emoticon)
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,Sangat Membantu. Terutama di kondisi Pandemi. ...,Pelayanan,Negatif
1,5,Baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,1,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,5,Terimakasih Halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,1,Ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


# CaseFolding

In [11]:
raw_clean['review'] = raw_clean['review'].apply(lambda x: x.lower())
raw_clean.head(100)

,rating,review,aspect_category,polarity
0,3,sangat membantu. terutama di kondisi pandemi. ...,Pelayanan,Negatif
1,5,baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,1,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,5,terimakasih halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,1,ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif
...,...,...,...,...
95,5,"aplikasi ini membuat dokter spt keluarga, saha...",Anekdot/Serbaneka,Positif
96,1,gimana sih ini. masa dokter nya yg ngelewatin ...,Pelayanan,Negatif
97,2,kemaren tepatnya hari kamis tgl 04-maret-20 20...,Anekdot/Serbaneka,Negatif
98,5,sangat membant untuk konsultasi dari rumah saj...,Pelayanan,Positif


# Filtering 

In [12]:
raw_clean['review'] = raw_clean['review'].apply(translate_non_alpha_num)
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,sangat membantu. terutama di kondisi pandemi. ...,Pelayanan,Negatif
1,5,baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,1,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,5,terimakasih halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,1,ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


In [13]:
raw_clean['review'] = raw_clean['review'].apply(remove_non_alphanumeric)
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,sangat membantu terutama di kondisi pandemi ...,Pelayanan,Negatif
1,5,baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,1,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,5,terimakasih halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,1,ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


In [14]:
#raw_clean['review'] = raw_clean['review'].apply(delete_suffix_nya)
#raw_clean.head()

In [15]:
raw_clean['review'] = raw_clean['review'].apply(normalizing_words)
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,sangat membantu terutama di kondisi pandemi ta...,Pelayanan,Negatif
1,5,baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,1,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,5,terimakasih halodoc atas pelayananya sangat ba...,Pelayanan,Positif
4,1,ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


In [16]:
def remove_numeric(review):
    return re.sub("\d", " ", review)

raw_clean['review'] = raw_clean['review'].apply(remove_numeric)

In [17]:
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,sangat membantu terutama di kondisi pandemi ta...,Pelayanan,Negatif
1,5,baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,1,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,5,terimakasih halodoc atas pelayananya sangat ba...,Pelayanan,Positif
4,1,ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


# Tokenizing

In [18]:
raw_clean['review'] = raw_clean['review'].apply(lambda x: x.split())
raw_clean['review'] = raw_clean['review'].apply(lambda x: np.array(x))
def mapping_slang_words(review):
    return [slang_words[word] if word in slang_words else word for word in review]
raw_clean['review'] = raw_clean['review'].apply(mapping_slang_words)
raw_clean.head(300)

,rating,review,aspect_category,polarity
0,3,"[sangat, membantu, terutama, di, kondisi, pand...",Pelayanan,Negatif
1,5,"[baru, pertama, konsultasi, dokter, lewat, hal...",Pelayanan,Negatif
2,1,"[astaga, obatnya, semakin, lama, beli, obat, d...",Harga,Negatif
3,5,"[terima kasih, halodoc, atas, pelayananya, san...",Pelayanan,Positif
4,1,"[ada, yang, menelepon, ke, nomor, saya, katany...",Sistem,Positif
...,...,...,...,...
295,5,"[dokternya, selalu, tangap, dan, cepat, respon]",Pelayanan,Positif
296,2,"[halo, doc, beberapa, waktu, lalu, call, servi...",Sistem,Negatif
297,5,"[dokternya, balasnya, cepat, dan, ramah]",Pelayanan,Positif
298,3,"[halo, doc, saya, barusan, dapat, telepon, men...",Anekdot/Serbaneka,Negatif


# Remove Stopword

In [19]:
stop_words = np.array(pd.read_csv("data/external/stopwords_ID.txt",
                        sep="\n", header=None).values)

In [20]:
stop_words = np.unique(stop_words)

In [21]:
def remove_stop_words(word_list):
    return [word for word in word_list if word not in stop_words]
raw_clean['review'] = raw_clean['review'].apply(remove_stop_words)
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,"[membantu, kondisi, pandemi, tolong, evaluasi,...",Pelayanan,Negatif
1,5,"[konsultasi, dokter, bagus, durasinya, cepat, ...",Pelayanan,Negatif
2,1,"[astaga, obatnya, beli, obat, harganya, masuk,...",Harga,Negatif
3,5,"[terima kasih, pelayananya, baik, cepat, pengi...",Pelayanan,Positif
4,1,"[menelepon, nomor, hadiah, konfirmasi, aplikas...",Sistem,Positif


In [22]:
raw['review'][0]

['membantu',
 'kondisi',
 'pandemi',
 'tolong',
 'evaluasi',
 'ketersedian',
 'obat',
 'apotek',
 'rekanan',
 'kota',
 'manado',
 'resep',
 'obat',
 'batuk',
 'apotek',
 'suka',
 'habis',
 'sia',
 'sia',
 'konsultasi',
 'dokter',
 'ujung',
 'ujungnya',
 'obat',
 'resep',
 'apotek',
 'rugi',
 'chat',
 'mahal',
 'biaya',
 'masukan',
 'baik',
 'pakai',
 'fitur',
 'chat',
 'us',
 'pelayanan',
 'pelayanan',
 'mengubah',
 'produk',
 'obat',
 'menyarankan',
 'konsultasi',
 'dokterjelek']

Contoh emoticon

In [23]:
raw['review'][784]

['terbaik', 'bagus', 'membantu', 'pandemi', 'suka']

In [24]:
raw['review'][101]

['terimakasih', 'bagus', 'panik', 'keluhan', 'bantu']

Contoh angka

In [25]:
raw['review'][424]

['pengguna',
 'bagus',
 'terima kasih',
 'penilaian',
 'berubah',
 'bagus',
 'pelayanan']

In [26]:
raw['review'][662]

['konsultasi', 'semoga', 'baik']

Contoh kata mengulang

In [27]:
raw['review'][790]

['dokternya', 'ramah', 'ramah', 'detail']

In [28]:
raw['review'][686]

['bicara', 'bicara', 'menyediakan', 'kalender', 'menstruasi']

Contoh translasi non alfanumerik

In [29]:
raw['review'][624]

['cepat', 'respon', 'dokter', 'ramah', 'penjelasan', 'detail']

In [30]:
raw['review'][621]

['membantu', 'tenang', 'informasi', 'solusi']

Contoh kata memiliki 'nya'

In [31]:
raw['review'][44]

['dokternya',
 'hebat',
 'hebat',
 'jelas',
 'fokus',
 'keluhan',
 'derita',
 'terima',
 'kasih']

In [32]:
raw['review'][47]

['respon',
 'keluhan',
 'tim',
 'pelayanan',
 'respon',
 'ulasan',
 'memperbarui',
 'respon',
 'tim',
 'pelayanan',
 'cepat']

In [33]:
raw['review'][56]

['membantu',
 'darurat',
 'langsung',
 'tindakan',
 'dokternya',
 'ramah',
 'terima kasih']

Contoh normalisasi kalimat

In [34]:
raw['review'][12]

['membantu',
 'jam',
 'anak',
 'sakit',
 'konsultasi',
 'dokter',
 'terima',
 'kasih',
 'adi',
 'suryadinata',
 'membantu']

In [35]:
raw['review'][13]

['membantu',
 'mempermudah',
 'mencari',
 'obat',
 'obatan',
 'respon',
 'cepat',
 'pesan',
 'terima kasih']

In [36]:
raw['review'][19]

['membantu',
 'obat',
 'dicari',
 'dibeberapa',
 'apotek',
 'kosong',
 'aplikasi',
 'lengkap',
 'pemakaian',
 'aplikasi',
 'mudah',
 'thanks']

contoh slang

In [37]:
raw['review'][23]

['bagus',
 'aplikasi',
 'ketemu',
 'kurangnya',
 'di mana',
 'bagus',
 'respon',
 'dokternya',
 'cepat',
 'bagus',
 'pertahankan',
 'membantu',
 'bagus']

In [38]:
raw['review'][18]

['jujur',
 'puas',
 'nasihat',
 'nasihat',
 'penjelasan',
 'dokternya',
 'disayangkan',
 'konsultasi',
 'aplikasi',
 'menit',
 'otomatis',
 'berhenti']

In [39]:
raw['review'][52]

['bagus',
 'antre',
 'sakit',
 'chat',
 'langsung',
 'dibalas',
 'obat',
 'diantar',
 'terima',
 'kasih',
 'bantu']

In [40]:
raw['review'][5]

['sumpah',
 'alternatif',
 'miris',
 'uang',
 'sakit',
 'dirasa',
 'hemat',
 'pakai',
 'aplikasi',
 'bagus',
 'pekerjaan',
 'cocok',
 'anak',
 'kosan']

In [41]:
raw_clean['polarity'] =raw_clean['polarity'].apply(lambda x: x.lower())
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,"[membantu, kondisi, pandemi, tolong, evaluasi,...",Pelayanan,negatif
1,5,"[konsultasi, dokter, bagus, durasinya, cepat, ...",Pelayanan,negatif
2,1,"[astaga, obatnya, beli, obat, harganya, masuk,...",Harga,negatif
3,5,"[terima kasih, pelayananya, baik, cepat, pengi...",Pelayanan,positif
4,1,"[menelepon, nomor, hadiah, konfirmasi, aplikas...",Sistem,positif


# Save

In [42]:
raw_clean.to_csv('data/data/data_clean.csv', index=False)
raw_clean.to_excel('data/data/data_clean_excel.xlsx', index=False)

In [43]:
raw_clean['review'][0]

['membantu',
 'kondisi',
 'pandemi',
 'tolong',
 'evaluasi',
 'ketersedian',
 'obat',
 'apotek',
 'rekanan',
 'kota',
 'manado',
 'resep',
 'obat',
 'batuk',
 'apotek',
 'suka',
 'habis',
 'sia',
 'sia',
 'konsultasi',
 'dokter',
 'ujung',
 'ujungnya',
 'obat',
 'resep',
 'apotek',
 'rugi',
 'chat',
 'mahal',
 'biaya',
 'masukan',
 'baik',
 'pakai',
 'fitur',
 'chat',
 'us',
 'pelayanan',
 'pelayanan',
 'mengubah',
 'produk',
 'obat',
 'menyarankan',
 'konsultasi',
 'dokterjelek']

In [44]:
raw_clean['review'][784]

['terbaik', 'bagus', 'membantu', 'pandemi', 'suka']

In [45]:
raw_clean['review'][101]

['terimakasih', 'bagus', 'panik', 'keluhan', 'bantu']

In [46]:
raw_clean['review'][424]

['pengguna',
 'bagus',
 'terima kasih',
 'penilaian',
 'berubah',
 'bagus',
 'pelayanan']

In [47]:
raw_clean['review'][662]

['konsultasi', 'semoga', 'baik']

In [48]:
raw_clean['review'][790]

['dokternya', 'ramah', 'ramah', 'detail']

In [49]:
raw_clean['review'][686]

['bicara', 'bicara', 'menyediakan', 'kalender', 'menstruasi']

In [50]:
raw_clean['review'][624]

['cepat', 'respon', 'dokter', 'ramah', 'penjelasan', 'detail']

In [51]:
raw_clean['review'][621]

['membantu', 'tenang', 'informasi', 'solusi']

In [52]:
raw_clean['review'][44]

['dokternya',
 'hebat',
 'hebat',
 'jelas',
 'fokus',
 'keluhan',
 'derita',
 'terima',
 'kasih']

In [53]:
raw_clean['review'][47]

['respon',
 'keluhan',
 'tim',
 'pelayanan',
 'respon',
 'ulasan',
 'memperbarui',
 'respon',
 'tim',
 'pelayanan',
 'cepat']

In [54]:
raw_clean['review'][56]

['membantu',
 'darurat',
 'langsung',
 'tindakan',
 'dokternya',
 'ramah',
 'terima kasih']

In [55]:
raw_clean['review'][12]

['membantu',
 'jam',
 'anak',
 'sakit',
 'konsultasi',
 'dokter',
 'terima',
 'kasih',
 'adi',
 'suryadinata',
 'membantu']

In [56]:
raw_clean['review'][13]

['membantu',
 'mempermudah',
 'mencari',
 'obat',
 'obatan',
 'respon',
 'cepat',
 'pesan',
 'terima kasih']

In [57]:
raw_clean['review'][19]

['membantu',
 'obat',
 'dicari',
 'dibeberapa',
 'apotek',
 'kosong',
 'aplikasi',
 'lengkap',
 'pemakaian',
 'aplikasi',
 'mudah',
 'thanks']

In [58]:
raw_clean['review'][23]

['bagus',
 'aplikasi',
 'ketemu',
 'kurangnya',
 'di mana',
 'bagus',
 'respon',
 'dokternya',
 'cepat',
 'bagus',
 'pertahankan',
 'membantu',
 'bagus']

In [59]:
raw_clean['review'][18]

['jujur',
 'puas',
 'nasihat',
 'nasihat',
 'penjelasan',
 'dokternya',
 'disayangkan',
 'konsultasi',
 'aplikasi',
 'menit',
 'otomatis',
 'berhenti']

In [60]:
raw_clean['review'][52]

['bagus',
 'antre',
 'sakit',
 'chat',
 'langsung',
 'dibalas',
 'obat',
 'diantar',
 'terima',
 'kasih',
 'bantu']

In [61]:
raw_clean['review'][5]

['sumpah',
 'alternatif',
 'miris',
 'uang',
 'sakit',
 'dirasa',
 'hemat',
 'pakai',
 'aplikasi',
 'bagus',
 'pekerjaan',
 'cocok',
 'anak',
 'kosan']

In [62]:
raw_clean['review'][47]

['respon',
 'keluhan',
 'tim',
 'pelayanan',
 'respon',
 'ulasan',
 'memperbarui',
 'respon',
 'tim',
 'pelayanan',
 'cepat']

In [63]:
def csv_string_to_list(csv_string):
    return csv_string[1:-1].split()

def string_without_quotes(word_list):
    new = []
    for word in word_list:
        new.append(word.replace("'",""))
    return new

raw_clean['review'] = raw_clean['review'].apply(string_without_quotes)
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,"[membantu, kondisi, pandemi, tolong, evaluasi,...",Pelayanan,negatif
1,5,"[konsultasi, dokter, bagus, durasinya, cepat, ...",Pelayanan,negatif
2,1,"[astaga, obatnya, beli, obat, harganya, masuk,...",Harga,negatif
3,5,"[terima kasih, pelayananya, baik, cepat, pengi...",Pelayanan,positif
4,1,"[menelepon, nomor, hadiah, konfirmasi, aplikas...",Sistem,positif


In [64]:
raw_clean.aspect_category=raw_clean.aspect_category.astype(str)


In [65]:
raw_clean['aspect_category'] = raw_clean['aspect_category'].apply(lambda x: x.lower())
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,"[membantu, kondisi, pandemi, tolong, evaluasi,...",pelayanan,negatif
1,5,"[konsultasi, dokter, bagus, durasinya, cepat, ...",pelayanan,negatif
2,1,"[astaga, obatnya, beli, obat, harganya, masuk,...",harga,negatif
3,5,"[terima kasih, pelayananya, baik, cepat, pengi...",pelayanan,positif
4,1,"[menelepon, nomor, hadiah, konfirmasi, aplikas...",sistem,positif


In [66]:
raw_clean.groupby('aspect_category').size().sort_values(ascending=False)

aspect_category
pelayanan            396
anekdot/serbaneka    302
sistem                83
harga                 18
harga                  1
dtype: int64

In [67]:
raw_clean['aspect_category'] = raw_clean['aspect_category'].apply(lambda x: x.strip())

In [68]:
raw_clean.groupby('aspect_category').size().sort_values(ascending=False)

aspect_category
pelayanan            396
anekdot/serbaneka    302
sistem                83
harga                 19
dtype: int64

In [69]:
def convert_list_to_string(word_list):
    return ",".join(word_list)

raw_clean['review'] = raw_clean['review'].apply(convert_list_to_string)

In [70]:
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,"membantu,kondisi,pandemi,tolong,evaluasi,keter...",pelayanan,negatif
1,5,"konsultasi,dokter,bagus,durasinya,cepat,dokter...",pelayanan,negatif
2,1,"astaga,obatnya,beli,obat,harganya,masuk,akal,l...",harga,negatif
3,5,"terima kasih,pelayananya,baik,cepat,pengiriman...",pelayanan,positif
4,1,"menelepon,nomor,hadiah,konfirmasi,aplikasi,men...",sistem,positif


In [71]:
raw_clean['polarity'] = raw_clean['polarity'].apply(lambda x: x.lower())
raw_clean.head()

,rating,review,aspect_category,polarity
0,3,"membantu,kondisi,pandemi,tolong,evaluasi,keter...",pelayanan,negatif
1,5,"konsultasi,dokter,bagus,durasinya,cepat,dokter...",pelayanan,negatif
2,1,"astaga,obatnya,beli,obat,harganya,masuk,akal,l...",harga,negatif
3,5,"terima kasih,pelayananya,baik,cepat,pengiriman...",pelayanan,positif
4,1,"menelepon,nomor,hadiah,konfirmasi,aplikasi,men...",sistem,positif


In [72]:
raw_clean.to_csv('data/data/data_clean_pure.csv', index=False)
raw_clean.to_excel('data/data/data_clean_excel_pure.xlsx', index=False)

In [73]:
raw_clean.groupby('polarity').size().sort_values(ascending=False)

polarity
positif    650
negatif    150
dtype: int64

In [74]:
raw_clean['review'][52]

'bagus,antre,sakit,chat,langsung,dibalas,obat,diantar,terima,kasih,bantu'

In [75]:
raw_clean['review'][662]

'konsultasi,semoga,baik'

In [76]:
raw_clean['review'][208]

'aplikasi,berguna,efisien,memesan,obat,dimanapun,cari,apotek,menyediakan,obatnya,suka,cinta,aplikasi,kembangkan'

In [77]:
raw_clean['review'][425]

'bagus,dapetin,obatnya,repot,apotek,terima kasih,bagus,konsultasi,dokter,lancar,cepat'

In [78]:
raw_clean['review'][208]

'aplikasi,berguna,efisien,memesan,obat,dimanapun,cari,apotek,menyediakan,obatnya,suka,cinta,aplikasi,kembangkan'